In [17]:
import numpy as np
import pandas as pd
import pyautogui
import pyperclip
import time
import tkinter as tk
from openpyxl import load_workbook
import undetected_chromedriver as uc
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from tabulate import tabulate
from webdriver_manager.chrome import ChromeDriverManager
servico = Service(ChromeDriverManager().install())


In [18]:
edge=pyautogui.locateCenterOnScreen('imagensAut/edge.png', confidence=0.8)
pyautogui.click(edge)
time.sleep(3)
sitePlan = 'https://anatel365.sharepoint.com/:x:/r/sites/lista.orcn/_layouts/15/Doc.aspx?sourcedoc=%7B8815BAD7-DE9F-4086-8024-39109264EEAE%7D&file=Distribui%C3%A7%C3%A3o%20Processo%20Drone.xlsx&action=default&mobileredirect=true&wdsle=0'
pyperclip.copy(sitePlan)
pyautogui.hotkey('ctrl', 'v')
pyautogui.press('enter')

In [ ]:
#INICIA O NAVEGADOR
navegador = uc.Chrome(service=servico)
navegador.maximize_window()
navegador.get('https://sei.anatel.gov.br/')


In [3]:
#FUNCAO QUE RECEBE USUARIO E SENHA
def preencher_campos():
    user = user_var.get()
    senha = senha_var.get()

    navegador.find_element(By.XPATH,'//*[@id="txtUsuario"]').send_keys(f"{user}")
    time.sleep(1)
    navegador.find_element(By.XPATH,'//*[@id="pwdSenha"]').send_keys(f"{senha}")
    root.destroy()

#INICIA JANELA
root = tk.Tk()
root.title("Login de usuário")
root.attributes('-topmost', True)
largura_tela = root.winfo_screenwidth()
root.geometry(f'+{largura_tela}+0')
root.state('zoomed')

#RECEBE USUARIO E SENHA
user_var = tk.StringVar()
senha_var = tk.StringVar()

frame = tk.Frame(root)
frame.place(relx=0.5, rely=0.5, anchor='center')

# CRIA LABEL CENTRALIZADO
tk.Label(frame, text="LOGIN NO SEI ANATEL", font=('Arial',30)).grid(row=0, columnspan=2)
tk.Label(frame, text="").grid(row=1, column=0)
tk.Label(frame, text="").grid(row=2, column=0)

#CRIA INPUT USUARIO
tk.Label(frame, text="Usuário:", font=('Arial',15)).grid(row=3, column=0, sticky='e')
tk.Entry(frame, textvariable=user_var,width=30).grid(row=3, column=1, sticky='w')

#CRIA INPUT SENHA CENSURADA
tk.Label(frame, text="Senha:", font=('Arial',15)).grid(row=4, column=0, sticky='e')
tk.Entry(frame, textvariable=senha_var, show="*",width=30).grid(row=4, column=1, sticky='w')

#BOTAO QUE ENVIA USUARIO E SENHA
tk.Label(frame, text="").grid(row=5, column=0)
tk.Button(frame, text="Entrar", font=('Arial',10), command=preencher_campos, width=20).grid(row=6, columnspan=2)

#INFORMA QUE OS DADOS NAO SERAO ARMAZENADOS NO SOFTWARE
tk.Label(frame, text="").grid(row=7, column=0)
tk.Label(frame, text="").grid(row=8, column=0)
tk.Label(frame, text="O usuário e senha não serão armazenados em nenhum local durante a execução do script. Ambos serão utilizados apenas para o acesso ao site.", font=('Arial',15)).grid(row=9, columnspan=2)

label = tk.Label(root, text="Script made by Theo Ravi.", font=('Arial', 10))
label.place(relx=1.0, rely=1.0, anchor='se')

root.mainloop()
#ACESSA O SEI
time.sleep(1)
navegador.find_element(By.XPATH,'//*[@id="sbmAcessar"]').click()    
# ENTRAR NO PROCESSO
navegador.find_element(By.XPATH,'//*[@id="divFiltro"]/div[2]/a').click()


In [4]:
tbody = navegador.find_element(By.XPATH,'//*[@id="tblProcessosRecebidos"]/tbody')
trs = tbody.find_elements(By.TAG_NAME, 'tr')
lista_processos = []
lista_procConformes = []
for tr in trs:
    tds = tr.find_elements(By.TAG_NAME, 'td')
    for td in tds:
        texto = td.text
        if '.estagio' not in texto:
            texto = texto.replace('Recebidos', '')
            texto = texto.replace(' ', '')
            anotacoes = tr.find_elements(By.XPATH, './/img[@src="svg/anotacao1.svg?11"]')
            if not anotacoes:  # se a lista de anotações estiver vazia, o elemento não existe
                lista_processos.append(texto)
            else:
                lista_procConformes.append(texto)

lista_processos = [processo for processo in lista_processos if processo]
lista_processos.reverse()
lista_procConformes = [processo for processo in lista_procConformes if processo]
lista_procConformes.reverse()

print(lista_procConformes, lista_processos)
print("Quantidade de processos para analisar",len(lista_processos))
print("Quantidade de processos para concluir", len(lista_procConformes))

[] ['53500.046023/2024-71']
Quantidade de processos para analisar 1
Quantidade de processos para concluir 0


In [5]:
planilhaDrones = input("Insira o caminho da planilha de drones: ")
planilhaDrones = planilhaDrones.replace('"' , '')
tabela = pd.read_excel(planilhaDrones)
tabela.columns = tabela.iloc[1]
tabela = tabela.iloc[2:]# Cria uma nova lista de cabeçalhos
tabela = tabela.reset_index(drop=True)


In [6]:
#FUNCAO QUE VERIFICA SE UM ELEMENTO EXISTE NA TELA 
def check_element_exists(by, value):
    try:
        navegador.find_element(by, value)
        return True
    except NoSuchElementException:
        return False
    

In [7]:
#FUNCAO PARA INSERIR ANEXOS
def insira_anexo(processos):
    exig1 = input("Insira apenas a exigência que seria colocada no corpo do email: ")
    navegador.find_element(By.ID, 'txtAssunto').send_keys(f"Processo SEI nº {processos} - Exigência")
    navegador.find_element(By.ID, 'txaMensagem').send_keys(f'''Prezado(a) Senhor(a),

    Em atenção à demanda registrada no processo em referência, apresentamos as seguintes pendências observadas durante a análise da solicitação:

    {exig1}

    Favor seguir as seguintes orientações:
    
    1. Solicitamos acessar o sistema Sei da Anatel: https://sei.anatel.gov.br/sei/controlador_externo.php?acao=usuario_externo_logar&id_orgao_acesso_externo=0    

    2. Selecionar o processo e clicar em Peticionamento Intecorrente;

    3. Selecionar os arquivos que deseja incluir para atender à(s) exigência(s), selecionar o Tipo do Documento, Nível de Acesso e Formato. Após isso clique em Adicionar.

    4. Clique em Peticionar.

    5. Indicar o Cargo/Função e senha de acesso ao SEI. Clique em Assinar.



    FAVOR NÃO RESPONDER ESTE E-MAIL.



    Atenciosamente,

    ORCN - Gerência de Certificação e Numeração

    SOR - Superintendência de Outorga e Recursos à Prestação

    Anatel - Agência Nacional de Telecomunicações''')

#FUNCAO PARA ERROS NA DECLARACAO DE CONFORMIDADE PEDINDO PARA ABRIR NOVO PROCESSO SEI
def erro_declaracao(processos):
    exig2 = input("Insira apenas a exigência que seria colocado no corpo do email: ")
    navegador.find_element(By.ID, 'txtAssunto').send_keys(f"Processo SEI nº {processos} - Indeferido")
    navegador.find_element(By.ID, 'txaMensagem').send_keys(f'''Prezado(a) Senhor(a),

    Em atenção à demanda registrada no processo em referência, apresentamos as seguintes pendências observadas no documento de Declaração de Conformidade:

    {exig2}

    Favor seguir as seguintes orientações:
    
    1.  Realizar um novo processo SEI, cumprindo com a(s) exigência(s) apontada(s) anteriormente. Seguindo o manual: https://docs.google.com/document/d/1YsrMwMxyVysCJ9VhtGOOU1S1FlbmkRFaYSjojLxXVsQ/edit

    2. Para criar um novo processo SEI, solicitamos acessar o sistema Sei da Anatel: https://sei.anatel.gov.br/sei/controlador_externo.php?acao=usuario_externo_logar&id_orgao_acesso_externo=0

    Dessa maneira, devido a presença de exigência(s), informamos que o atual processo SEI nº {processos} será arquivado.

    FAVOR NÃO RESPONDER ESTE E-MAIL.



    Atenciosamente,

    ORCN - Gerência de Certificação e Numeração

    SOR - Superintendência de Outorga e Recursos à Prestação

    Anatel - Agência Nacional de Telecomunicações''')

#FUNCAO QUE PERMITE O USUARIO INSERIR CORPO DO EMAIL
def outro_erro():
    assunto  = input("Insira o assunto do email: ")
    exig3 = input("Insira a mensagem por inteiro (incluindo corpo do email): ")
    navegador.find_element(By.ID, 'txtAssunto').send_keys(assunto)
    navegador.find_element(By.ID, 'txaMensagem').send_keys(exig3)

#FUNCAO QUE ANALISA PROCESSO E CRIA DESPACHO DECISORIO OU GERA EXIGENCIA
def analisaProcesso():
    nomeEstag=input('Insira seu nome completo: ')
    janela_principal = navegador.current_window_handle
    janela_principal = navegador.current_window_handle
    for processos in lista_processos:
        while True:
            analisar = int(input("Deseja analisar? Se sim digite [1], se não digite [2]: "))
            if analisar == 2:
                return
            else:
                try:    
                    #PESQUISA NUMERO DO PROCESSO PARA ACESSAR SUA PAGINA
                    navegador.switch_to.default_content()
                    navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processos) 
                    elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                    elementos.send_keys(Keys.ENTER)    
                    time.sleep(1)

                    #ENCONTRAR DECLARAÇÃO DE CONFORMIDADE NO PROCESSO DRONE
                    navegador.switch_to.frame('ifrArvore')
                    if check_element_exists(By.PARTIAL_LINK_TEXT, 'Despacho Decisório'):
                        print(f"O processo {processos} já foi despachado!")
                    elif check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Drone") or check_element_exists(By.PARTIAL_LINK_TEXT, "I"):
                        if check_element_exists(By.PARTIAL_LINK_TEXT, "I"):
                            navegador.find_element(By.PARTIAL_LINK_TEXT, "I").click()
                            time.sleep(1)
                        #PEGAR NOME DO SOLICITANTE NO RECIBO ELETRONICO
                        navegador.find_element(By.PARTIAL_LINK_TEXT, 'Recibo Eletrônico').click()
                        navegador.switch_to.default_content()
                        time.sleep(1)
                        navegador.switch_to.frame('ifrVisualizacao')
                        navegador.switch_to.frame('ifrArvoreHtml')
                        nomeSol = navegador.find_element(By.XPATH, '//*[@id="conteudo"]/table/tbody/tr[1]/td[2]').text
                        navegador.switch_to.default_content()
                        #COLETAR DADOS DA DECLARAÇÃO DE CONFORMIDADE
                        navegador.switch_to.frame('ifrArvore')
                        navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Drone").click()
                        time.sleep(1)
                        navegador.switch_to.default_content()
                        navegador.switch_to.frame('ifrVisualizacao')
                        navegador.switch_to.frame('ifrArvoreHtml')
                        #EXIBE INFORMACOES DO PROCESSO
                        print("--------------------------------------------------------------------------")
                        print(f"Processo: {processos}")

                        #COLETAR TEXTO NOME DO INTERESSADO
                        nome_interessado = navegador.find_element(By.XPATH, '/html/body/table[1]/tbody/tr/td').text
                        print("Interessado: ",nome_interessado)
                        print("Solicitante: ",nomeSol)
                        print('\n')

                        #CHECKBOX DE QUEM ESTÁ FAZENDO O PETICIONAMENTO
                        checkbox=navegador.find_element(By.XPATH,'/html/body/table[2]/tbody/tr[1]/td[1]').text
                        checkbox2=navegador.find_element(By.XPATH, '/html/body/table[2]/tbody/tr[2]/td[1]').text
                        #CHECA CAMPO DA CHECKBOX
                        if not checkbox.strip() and not checkbox2.strip():
                            print("O usuário não informou quem está fazendo o peticionamento, confira os nomes do documento.")
                        elif not checkbox.strip():
                            procuracao_eletronica = navegador.find_element(By.XPATH, '/html/body/table[2]/tbody/tr[2]/td[3]').text
                            print("Peticionamento representando terceiro, Pessoa Jurídica ou Pessoa Física.", f"\nProcuração eletrônica: {procuracao_eletronica}.")
                        else:
                            print("Peticionamento em interesse próprio como Pessoa Física.")
                        print('\n')

                        #CONTA QUANTIDADE DE LINHAS DA TABELA SOBRE O PRODUTO
                        quantidade_linhas = len(navegador.find_elements(By.XPATH, '/html/body/table[3]/tbody/tr'))

                        #VERIFICA CAMPOS QUE CONTÉM AS INFORMAÇÕES SOBRE O PRODUTO
                        modelos = []
                        for i in range(1, quantidade_linhas, 1):
                            linha = navegador.find_element('xpath', f'/html/body/table[3]/tbody/tr[{i}]')
                            celulas = linha.find_elements(By.TAG_NAME,'td')
                            modelos_produto = [celula.text for celula in celulas]
                            modelos.append(modelos_produto)

                        #CRIA DATAFRAME PARA PRINTAR AS INFORMAÇÕES DO PRODUTO
                        df = pd.DataFrame(modelos)
                        df.columns = ['Modelo', 'Nome Comercial', 'Número de Série (incluindo rádio controle e óculos)']
                        df = df[1:]
                        df = df.replace(' ', np.nan)
                        df = df.dropna(how='all')

                        data = df.values.tolist()
                        headers = df.columns.tolist()
                        print(tabulate(data, headers=headers, tablefmt='pretty'))
                        print('\n')
                        
                        #VERIFICA SE O MODELO DO DRONE E RADIO CONTROLE ESTA NA PLANILHA DE DRONES CONFORMES
                        modelos = df['Modelo']
                        modelos = modelos.reset_index(drop=True)
                        checkexcel = modelos.isin(tabela['MODELO'])
                        for i in range(len(checkexcel)):
                            if checkexcel[i] == True:
                                print(f"O modelo {modelos[i]} está na lista de drones conformes.")
                            else:
                                print(f"O modelo {modelos[i]} não se encontra na lista de drones conformes")
                        print('\n')
                        #EXIBE CÓDIGO DE RASTREIO
                        #LINHA ESPECIFICA ESCREVE NA PLANILHA SE O PRODUTO ESTA RETIDO E, CASO ESTEJA, ESCREVE O CODIGO DE RASTREIO
                        codigo_rastreio = navegador.find_element(By.XPATH,'/html/body/table[4]/tbody/tr/td[2]').text
                        if not codigo_rastreio.strip():
                            print("Produto não retido ou código de rastreio não informado.")
                            retido = 'Não'
                        else:
                            print(f'O código de rastreio é: {codigo_rastreio}.')
                            retido = 'Sim'
                            
                        print('\n')
                        print('Confira o relatório fotográfico e veja se os documentos estão conformes!\n')
                        print('Preencha planilha no sharepoint sobre o processo.')
                        print('--------------------------------------------------------------------------')
                        
                    #ENCONTRAR DECLARAÇÃO DE CONFORMIDADE NO PROCESSO IMPORTADO USO PRÓPRIO
                    elif check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Importado Uso Próprio"):
                        #PEGAR NOME DO SOLICITANTE NO RECIBO ELETRONICO
                        navegador.find_element(By.PARTIAL_LINK_TEXT, 'Recibo Eletrônico').click()
                        navegador.switch_to.default_content()
                        time.sleep(0.5)
                        navegador.switch_to.frame('ifrVisualizacao')
                        navegador.switch_to.frame('ifrArvoreHtml')
                        nomeSol = navegador.find_element(By.XPATH, '//*[@id="conteudo"]/table/tbody/tr[1]/td[2]').text
                        navegador.switch_to.default_content()
                        #COLETAR DADOS DA DECLARAÇÃO DE CONFORMIDADE
                        navegador.switch_to.frame('ifrArvore')
                        navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Importado Uso Próprio").click()
                        time.sleep(0.5)
                        navegador.switch_to.default_content()
                        navegador.switch_to.frame('ifrVisualizacao')
                        navegador.switch_to.frame('ifrArvoreHtml')

                        #EXIBE INFORMACOES DO PROCESSO
                        print("--------------------------------------------------------------------------")
                        print(f"Processo: {processos}")

                        #COLETAR TEXTO NOME DO INTERESSADO
                        nome_interessado=navegador.find_element(By.XPATH, '/html/body/table[1]/tbody/tr/td/p').text
                        print(nome_interessado)
                        print(nomeSol)
                        print('\n')

                        #CHECKBOX DE QUEM ESTÁ FAZENDO O PETICIONAMENTO
                        checkbox=navegador.find_element('xpath','/html/body/table[2]/tbody/tr[1]/td[1]/p').text
                        checkbox2=navegador.find_element(By.XPATH, '/html/body/table[2]/tbody/tr[2]/td[1]').text

                        #CHECA CAMPO DA CHECKBOX
                        if not checkbox.strip() and not checkbox2.strip():
                            print("O usuário não informou quem está fazendo o peticionamento, confira os nomes do documento.")
                        elif not checkbox.strip():
                            procuracao_eletronica = navegador.find_element(By.XPATH, '/html/body/table[2]/tbody/tr[2]/td[3]').text
                            print("Peticionamento representando terceiro, Pessoa Jurídica ou Pessoa Física.", f"\nProcuração eletrônica: {procuracao_eletronica}.")
                        else:
                            print("Peticionamento em interesse próprio como Pessoa Física.")
                        print('\n')
                        #INFORMA O TIPO DE PRODUTO QUE ESTÁ SENDO TRATADO
                        print("\nTipo de produto:")
                        quantidade_linhasPro = len(navegador.find_elements(By.XPATH, '/html/body/table[3]/tbody/tr'))
                        tipoPro = []
                        for i in range(1, quantidade_linhasPro, 1):
                            linhas = navegador.find_element(By.XPATH, f'/html/body/table[3]/tbody/tr[{i}]')
                            celulas = linhas.find_elements(By.TAG_NAME,'td')
                            produtos = [celula.text for celula in celulas]
                            tipoPro.append(produtos)
                        dfPro = pd.DataFrame(tipoPro)
                        dfPro.columns = ['Tipo de Equipamento', 'Assinale com um X', 'Observação:']
                        dfPro = dfPro.drop('Observação:', axis=1)
                        dfPro = dfPro[1:]

                        data = dfPro.values.tolist()
                        headers = dfPro.columns.tolist()
                        print(tabulate(data, headers=headers, tablefmt='pretty'))
                        print('\n')

                        #CONTA QUANTIDADE DE LINHAS DA TABELA SOBRE O PRODUTO
                        quantidade_linhas = len(navegador.find_elements(By.XPATH, '/html/body/table[4]/tbody/tr'))

                        #VERIFICA CAMPOS QUE CONTÉM AS INFORMAÇÕES SOBRE O PRODUTO
                        modelos = []
                        for i in range(1, quantidade_linhas, 1):
                            linha = navegador.find_element('xpath', f'/html/body/table[4]/tbody/tr[{i}]')
                            celulas = linha.find_elements(By.TAG_NAME,'td')
                            modelos_produto = [celula.text for celula in celulas]
                            modelos.append(modelos_produto)
                        print("Modelos:")
                        #CRIA DATAFRAME PARA PRINTAR AS INFORMAÇÕES DO PRODUTO
                        df = pd.DataFrame(modelos)
                        df.columns = ['Modelo', 'Nome Comercial', 'Número de Série']
                        df = df[1:]
                        df = df.replace(' ', np.nan)
                        df = df.dropna(how='all')

                        data2 = df.values.tolist()
                        headers2 = df.columns.tolist()
                        print(tabulate(data2, headers=headers2, tablefmt='pretty'))
                        print('\n')

                        modelos = df['Modelo']
                        modelos = modelos.reset_index(drop=True)
                        checkexcel = modelos.isin(tabela['MODELO'])
                        for i in range(len(checkexcel)):
                            if checkexcel[i] == True:
                                print(f"O modelo {modelos[i]} está na lista de drones conformes.")
                            else:
                                print(f"O modelo {modelos[i]} não se encontra na lista de drones conformes")
                        print('\n')
                        #EXIBE CÓDIGO DE RASTREIO
                        #ESCREVE NA TABELA EXCEL SE ESTA RETIDO, CASO ESTEJA INSERE CODIGO DE RASTREIO
                        codigo_rastreio = navegador.find_element(By.XPATH,'/html/body/table[5]/tbody/tr/td[2]').text
                        if not codigo_rastreio.strip():
                            print("Produto não retido ou código de rastreio não informado.")
                            retido = 'Não'
                        else:
                            print(f'O código de rastreio é: {codigo_rastreio}.')
                            retido = 'Sim'
                        print('\n')
                        print('Confira o relatório fotográfico e veja se os documentos estão conformes!\n')
                        print('Preencha planilha no sharepoint sobre o processo.')
                        print('--------------------------------------------------------------------------')
                        
                    time.sleep(1)
                    navegador.switch_to.default_content()
                    #CRIA DESPACHO DECISORIO
                    #PEDE PRA USUARIO VERIFICAR SE ESTÁ TUDO CERTO
                    #VOLTA PARA A PAGINA DE INICIO DO PROCESSO

                    confirmacao = input('Caso esteja tudo certo, digite "y" para continuar, senão, digite "n": ')
                    if confirmacao == 'y':
                        navegador.switch_to.frame('ifrArvore')
                        arvoredocs = navegador.find_elements(By.CLASS_NAME, 'infraArvoreNo')
                        elementos_com_src = navegador.find_elements(By.CSS_SELECTOR, "[src='svg/processo_restrito.svg?11']")
                        # Crie uma lista para armazenar os identificadores únicos dos elementos que você deseja clicar
                        elementos_para_clicar = []
                        for doc in arvoredocs:
                            # Itere sobre todos os elementos com o src desejado
                            for elemento in elementos_com_src:
                                # Verifique se o elemento é um irmão do documento
                                if elemento in doc.find_elements(By.XPATH, 'following-sibling::*[2]/*'):
                                    # Se o elemento é um irmão do documento, adicione o identificador único do documento à lista
                                    elementos_para_clicar.append(doc.get_attribute("id"))
                                    break  # Saia do loop interno uma vez que encontramos o elemento desejado
                        # Agora, iteramos sobre os identificadores únicos dos elementos que queremos clicar
                        for id in elementos_para_clicar:
                            # Encontramos o elemento novamente antes de clicar nele
                            doc = navegador.find_element(By.ID, id)
                            doc.click()
                            time.sleep(0.7)
                            navegador.switch_to.default_content()
                            navegador.switch_to.frame('ifrVisualizacao')
                            navegador.find_element(By.XPATH,'//*[@id="divArvoreAcoes"]/a[2]').click()
                            navegador.find_element(By.XPATH,'//*[@id="divOptPublico"]/div/label').click()
                            navegador.find_element(By.ID,'btnSalvar').click()
                            navegador.switch_to.default_content()
                            navegador.switch_to.frame('ifrArvore')
                        navegador.switch_to.default_content()
                        navegador.switch_to.frame('ifrArvore')
                        #ENTRA NA PAGINA INICIAL DO PROCESSO
                        if check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Drone"):
                            navegador.switch_to.default_content()
                            navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processos)
                            elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                            elementos.send_keys(Keys.ENTER)
                            #CRIA DESPACHO
                            navegador.switch_to.frame('ifrVisualizacao')
                            time.sleep(1)
                            navegador.find_element(By.XPATH,'//*[@id="divArvoreAcoes"]/a[1]').click()
                            time.sleep(1)
                            navegador.find_element(By.XPATH,'//*[@id="tblSeries"]/tbody/tr[16]/td/a[2]').click()
                            #SELECIONA TEXTO PADRAO
                            navegador.find_element(By.XPATH,'//*[@id="divOptTextoPadrao"]/div').click()
                            navegador.find_element(By.XPATH,'//*[@id="txtTextoPadrao"]').send_keys('Despacho Decisório de Homologação Drones')
                        elif check_element_exists(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade - Importado Uso Próprio"):
                            navegador.switch_to.default_content()
                            navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processos)
                            elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                            elementos.send_keys(Keys.ENTER)
                            #CRIA DESPACHO
                            navegador.switch_to.frame('ifrVisualizacao')
                            time.sleep(1)
                            navegador.find_element(By.XPATH,'//*[@id="divArvoreAcoes"]/a[1]').click()
                            time.sleep(1)
                            navegador.find_element(By.XPATH,'//*[@id="tblSeries"]/tbody/tr[16]/td/a[2]').click()
                            #SELECIONA TEXTO PADRAO
                            navegador.find_element(By.XPATH,'//*[@id="divOptTextoPadrao"]/div').click()
                            navegador.find_element(By.XPATH,'//*[@id="txtTextoPadrao"]').send_keys('Despacho Decisório de Homologação não licenciados')
                        time.sleep(2)
                        navegador.find_element(By.XPATH,'//*[@id="divInfraAjaxtxtTextoPadrao"]/ul/li/a').click()
                        #COLOCA O DESPACHO COMO PUBLICO
                        navegador.find_element(By.XPATH,'//*[@id="divOptPublico"]/div').click()
                        #SALVA DESPACHO
                        navegador.find_element(By.ID,'btnSalvar').click()
                        navegador.switch_to.window(navegador.window_handles[-1])
                        time.sleep(0.7)
                        navegador.close()
                        navegador.switch_to(janela_principal)
                        #INCLUI DESPACHO NO BLOCO
                        navegador.switch_to.default_content()
                        navegador.switch_to.frame('ifrArvore')
                        #SELECIONA DESPACHO DECISORIO
                        navegador.find_element(By.PARTIAL_LINK_TEXT, "Despacho Decisório").click()
                        navegador.switch_to.default_content()
                        navegador.switch_to.frame('ifrVisualizacao')
                        time.sleep(1)
                        #CLICA NO LEGO
                        navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[8]').click()
                        #SELECIONA BLOCO (ELE SELECIONA O PRIMEIRO DESPACHO PARA DRONES APROVADOS QUE LER)
                        select_element = navegador.find_element(By.ID, 'selBloco')
                        select = Select(select_element)
                        for opcao in select.options:
                            if "Despachos para Drones aprovados" in opcao.text:
                                select.select_by_visible_text(opcao.text)
                                break
                        navegador.find_element(By.XPATH, '//*[@id="sbmIncluir"]').click()
                        #VOLTA PARA PAGINA INICIAL DO PROCESSO
                        navegador.switch_to.default_content()
                        navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processos)
                        elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                        elementos.send_keys(Keys.ENTER)
                        navegador.switch_to.frame('ifrVisualizacao')
                        time.sleep(1)
                        #ADICIONA NOTA PARA AGUARDAR ASSINATURA
                        navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[17]').click()
                        navegador.find_element(By.ID, 'txaDescricao').send_keys('Aguardando assinatura.')
                        navegador.find_element(By.NAME, 'sbmRegistrarAnotacao').click()
                        situacao = 'Aprovado'
                        with open('processosConformes.txt', 'a') as arquivo2:
                            arquivo2.write(processos + '\n')
                        confere2 = int(input("Confira se está tudo certo e faça as alterações necessárias, logo em seguida, digite [1]: "))
                        if confere2 == 1:
                            print("Próximo processo...")
                    else:
                        print(f'Tome as medidas necessárias para o processo {processos}.')
                        #MOSTRA OPCOES DE EXIGENCIA
                        exigencia = input('Se houver alguma exigência digite "y", senão, digite "n".').lower()
                        if exigencia == 'y':
                            #PEGA ENDERECO DE EMAIL DO SOLICITANTE
                            navegador.switch_to.default_content()
                            navegador.switch_to.frame('ifrArvore')
                            navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade").click()
                            time.sleep(1)
                            navegador.switch_to.default_content()
                            navegador.switch_to.frame('ifrVisualizacao')
                            navegador.switch_to.frame('ifrArvoreHtml')
                            emailSol = navegador.find_element(By.XPATH, '/html/body/div[3]/a[1]').text
                            navegador.switch_to.default_content()
                            #RETORNA PARA A PAGINA INICIAL DO PROCESSO
                            navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processos)
                            elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                            elementos.send_keys(Keys.ENTER)
                            time.sleep(1)
                            #ABRE A TELA DE ENVIO DE EMAIL
                            navegador.switch_to.frame('ifrVisualizacao')
                            navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[11]').click()
                            time.sleep(1)
                            navegador.switch_to.window(navegador.window_handles[-1])
                            time.sleep(1)
                            select_element = navegador.find_element(By.XPATH, '//*[@id="selDe"]')
                            select = Select(select_element)
                            select.select_by_visible_text('ANATEL/E-mail de replicação <nao-responda@anatel.gov.br>')
                            navegador.find_element(By.XPATH, '//*[@id="s2id_autogen1"]').send_keys(emailSol)
                            time.sleep(0.5)
                            navegador.find_element(By.CLASS_NAME, 'select2-result-label').click()
                            #ESCREVE NUMERO DO PROCESSO NO TXT PROCESSOSERRO
                            with open('processosErro.txt', 'a') as arquivo3:
                                arquivo3.write(processos + '\n')
                            #MOSTRA AS OPCOES DE EXIGENCIA
                            print("Selecione o tipo de exigência abaixo:\n"
                                " [1] Falta algum anexo no processo.\n",
                                "[2] Erro na declaração de conformidade ou alguma exigência que é necessário abrir novo processo.\n",
                                "[3] Outros.")
                            time.sleep(0.5)
                            #EXECUTA A CONDICAO PARA CADA EXIGENCIA
                            while True:
                                try:
                                    exig = int(input("Opção: "))
                                except ValueError:
                                    print("Opção inválida. Digite um número inteiro.")
                                    continue
                                if exig in range(1, 3):
                                    if exig == 1:
                                        #PUXA FUNCAO INSIRA_ANEXO CONTENDO O CORPO DE EMAIL PEDINDO ANEXOS
                                        insira_anexo(processos)
                                        confere = input('Caso esteja tudo certo, digite "y", caso tenha algum erro digite "n": ')
                                        if confere == 'y':
                                            navegador.find_element(By.XPATH, '//*[@id="divInfraBarraComandosInferior"]/button[1]').click()
                                            navegador.switch_to.window(janela_principal)
                                            navegador.switch_to.default_content()
                                            navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processos) 
                                            elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                                            elementos.send_keys(Keys.ENTER)    
                                            time.sleep(1)
                                            #INSERE TAG REFERENTE A PROCESSOS INTERCORRENTES
                                            navegador.switch_to.frame('ifrVisualizacao') 
                                            navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[25]').click()
                                            time.sleep(0.5)
                                            navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
                                            time.sleep(0.5)
                                            navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/ul/li[18]/a').click()
                                            #PEDE O TEXTO DA TAG
                                            textoTag = input("Insira o texto da tag: ")
                                            navegador.find_element(By.XPATH, '//*[@id="txaTexto"]').send_keys(textoTag)
                                            navegador.find_element(By.XPATH, '//*[@id="sbmSalvar"]').click()
                                        else:
                                            print("Faça as alterações necessárias mais tarde.")
                                            navegador.find_element(By.XPATH, '//*[@id="btnCancelar"]').click()
                                    elif exig == 2:
                                        #PUXA FUNCAO ERRO_DECLARACAO CONTENDO O CORPO DE EMAIL PEDINDO PARA ABRIR NOVO PROCESSO SEI
                                        erro_declaracao(processos)
                                        confere = input('Caso esteja tudo certo, digite "y", caso tenha algum erro digite "n": ')
                                        if confere == 'y':
                                            navegador.find_element(By.XPATH, '//*[@id="divInfraBarraComandosInferior"]/button[1]').click()
                                            navegador.switch_to.window(janela_principal)
                                            navegador.switch_to.default_content()
                                            navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processos) 
                                            elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                                            elementos.send_keys(Keys.ENTER)    
                                            time.sleep(1)
                                            #INSERE TAG REFERENTE A PROCESSOS INTERCORRENTES 
                                            navegador.switch_to.frame('ifrVisualizacao')
                                            navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[25]').click()
                                            time.sleep(0.5)
                                            navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
                                            time.sleep(0.5)
                                            navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/ul/li[19]/a').click()
                                            #PEDE O TEXTO DA TAG
                                            textoTag = input("Insira o texto da tag: ")
                                            navegador.find_element(By.XPATH, '//*[@id="txaTexto"]').send_keys(textoTag)
                                            navegador.find_element(By.XPATH, '//*[@id="sbmSalvar"]').click()
                                        else:
                                            print("Faça as alterações necessárias mais tarde.")
                                            navegador.find_element(By.XPATH, '//*[@id="btnCancelar"]').click()
                                    elif exig == 3:
                                        #PUXA FUNCAO OUTRO_ERRO EM QUE USUARIO INSERE O EMAIL COMO QUISER
                                        '''POSSIVEL ALTERACAO NESSA CONDICAO'''
                                        outro_erro()
                                        confere = input('Caso esteja tudo certo, digite "y", caso tenha algum erro digite "n": ')
                                        if confere == 'y':
                                            navegador.find_element(By.XPATH, '//*[@id="divInfraBarraComandosInferior"]/button[1]').click()
                                            navegador.switch_to.window(janela_principal)
                                            navegador.switch_to.default_content()
                                            navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processos) 
                                            elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                                            elementos.send_keys(Keys.ENTER)    
                                            time.sleep(1)
                                            #INSERE TAG DE 'PENDENCIAS'
                                            navegador.switch_to.frame('ifrVisualizacao')
                                            navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[25]').click()
                                            time.sleep(0.5)
                                            navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
                                            time.sleep(0.5)
                                            navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/ul/li[19]/a').click()
                                            #PEDE TEXTO DA TAG
                                            textoTag = input("Insira o texto da tag: ")
                                            navegador.find_element(By.XPATH, '//*[@id="txaTexto"]').send_keys(textoTag)
                                            navegador.find_element(By.XPATH, '//*[@id="sbmSalvar"]').click()
                                        else:
                                            print("Faça as alterações necessárias mais tarde.")
                                            navegador.find_element(By.XPATH, '//*[@id="btnCancelar"]').click()
                                    #CONCLUI PROCESSO ELMINANDO-O DA CAIXA
                                    navegador.switch_to.default_content()
                                    navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processos) 
                                    elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                                    elementos.send_keys(Keys.ENTER)    
                                    time.sleep(1)
                                    navegador.switch_to.frame('ifrVisualizacao')
                                    navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[20]').click()
                                    situacao = 'Exigência'
                                    print("Próximo processo...")
                                    break
                                #CONDICAO PARA OPCAO INEXISTENTE
                                else:
                                    print("Opção inválida. Tente novamente.")
                            # FAZER DEPOIS MÉTODO PARA VERIFICAR OS PROCESSOS DO ARQUIVO PROCESSOSERROTXT
                        else:
                            print("Pulando processo.")

                    pyperclip.copy(processos)
                    pyautogui.click(edge)
                    time.sleep(3)
                    pyautogui.hotkey('ctrl','l')
                    pyautogui.hotkey('ctrl', 'v')
                    pyautogui.press('enter')
                    close=pyautogui.locateCenterOnScreen('imagensAut/fechar.png', confidence=0.7)
                    pyautogui.click(close)
                    time.sleep(0.5)
                    pyautogui.press('right')
                    pyautogui.write(nomeEstag)
                    pyautogui.press('right')
                    pyautogui.write(retido)
                    pyautogui.press('right')
                    if retido == 'Sim':
                        pyautogui.write(codigo_rastreio)
                    else:
                        pyautogui.press('right')
                    pyautogui.write(nomeSol)
                    pyautogui.press('right')
                    pyautogui.write(nome_interessado)
                    pyautogui.press('right')
                    pyautogui.write(situacao)
                except Exception as error:
                    print(f"Ocorreu um erro: {error}")                    
                break
                    

In [8]:
def concluiProcesso():
    textoRetido = '''Prezado(a) Senhor(a),

    Em atenção ao pedido de homologação constante do processo SEI em referência, informamos que:

    1. O pedido foi APROVADO
    2. O Despacho Decisório que aprovou o pedido está disponível publicamente por meio do sistema SEI na área de Pesquisa Pública, no link:

            https://sei.anatel.gov.br/sei/modulos/pesquisa/md_pesq_processo_pesquisar.php?acao_externa=protocolo_pesquisar&acao_origem_externa=protocolo_pesquisar&id_orgao_acesso_externo=0


    3. O Despacho Decisório deverá ser portado junto ao Equipamento (fisicamente ou eletronicamente), para que as autoridades competentes possam conferir a regularidade, quando necessário.
    4. Visto que o produto se encontra retido, para que seja informado o número de série, solicitamos que acesse o sistema SEI da Anatel por meio do seguinte link:


            https://sei.anatel.gov.br/sei/controlador_externo.php?acao=usuario_externo_logar&id_orgao_acesso_externo=0.


    5. Após autenticação no sistema SEI, solicitamos que inclua os documentos selecionando a opção "Intercorrente" e informando o número do processo em referência.



    FAVOR NÃO RESPONDER ESTE E-MAIL.

    Atenciosamente,

    ORCN - Gerência de Certificação e Numeração

    SOR - Superintendência de Outorga e Recursos à Prestação

    Anatel - Agência Nacional de Telecomunicações'''

    textoNaoRetido = '''Prezado(a) Senhor(a),

    Em atenção ao pedido de homologação constante do processo SEI em referência, informamos que:

    1. O pedido foi APROVADO
    2. O Despacho Decisório que aprovou o pedido está disponível publicamente por meio do sistema SEI na área de Pesquisa Pública, no link:

            https://sei.anatel.gov.br/sei/modulos/pesquisa/md_pesq_processo_pesquisar.php?acao_externa=protocolo_pesquisar&acao_origem_externa=protocolo_pesquisar&id_orgao_acesso_externo=0


    3. O Despacho Decisório deverá ser portado junto ao Equipamento (fisicamente ou eletronicamente), para que as autoridades competentes possam conferir a regularidade, quando necessário.

    FAVOR NÃO RESPONDER ESTE E-MAIL.

    Atenciosamente,

    ORCN - Gerência de Certificação e Numeração

    SOR - Superintendência de Outorga e Recursos à Prestação

    Anatel - Agência Nacional de Telecomunicações'''

    #VARIAVEL PARA VOLTAR À JANELA INICIAL
    janela_principal = navegador.current_window_handle

    #VERIFICA PROCESSOS DO TXT PROCESSOSCONFORMES
    
            #ITERA SOBRE OS PROCESSOS DENTRO DO TXT
    for processosAssinados in lista_procConformes:
        processosAssinados = processosAssinados.strip()
        while True:
            concluir = int(input("Deseja concluir um processo? Se sim digite [1], se não digite [2]: "))
            if concluir == 2:
                return
            else:
            #ENTRA NA PAGINA DO PROCESSO
                navegador.switch_to.default_content()
                navegador.find_element(By.ID,'txtPesquisaRapida').clear()
                navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processosAssinados)
                elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                elementos.send_keys(Keys.ENTER)
                time.sleep(1)
                #PEGA DADOS DA DECLARACAO DE CONFORMIDADE
                navegador.switch_to.frame('ifrArvore')
                navegador.find_element(By.PARTIAL_LINK_TEXT, "Declaração de Conformidade").click()
                time.sleep(1)
                navegador.switch_to.default_content()
                navegador.switch_to.frame('ifrVisualizacao')
                navegador.switch_to.frame('ifrArvoreHtml')
                tabela_cod = navegador.find_element(By.XPATH, '/html/body/table[4]/tbody/tr/td[1]').text
                #FLUXO PARA DRONES PRE APROVADOS PELA ANATEL 
                if check_element_exists(By.XPATH, '/html/body/table[4]/tbody/tr/td[2]') and tabela_cod == 'Código de rastreio':
                    codigo_rastreioConforme = navegador.find_element(By.XPATH, '/html/body/table[4]/tbody/tr/td[2]').text
                    emailSol = navegador.find_element(By.XPATH, '/html/body/div[3]/a[1]').text
                    time.sleep(0.5)
                    navegador.switch_to.default_content()
                    navegador.switch_to.frame('ifrArvore')
                    #VERIFICA ASSINATURA DO DESPACHO DECISORIO
                    if check_element_exists(By.PARTIAL_LINK_TEXT, "Despacho Decisório"):
                        navegador.find_element(By.PARTIAL_LINK_TEXT, "Despacho Decisório").click()
                        time.sleep(1)
                        navegador.switch_to.default_content()
                        navegador.switch_to.frame('ifrVisualizacao')
                        navegador.switch_to.frame('ifrArvoreHtml')
                        if check_element_exists(By.XPATH, "/html/body/div[1]/table[1]"):
                            #ENVIA EMAIL DE APROVACAO DO PROCESSO
                            navegador.switch_to.default_content()
                            navegador.switch_to.frame('ifrVisualizacao')
                            navegador.find_element(By.XPATH, "//img[@title='Enviar Documento por Correio Eletrônico']").click()
                            time.sleep(0.5)
                            navegador.switch_to.window(navegador.window_handles[-1])
                            time.sleep(1)
                            select_element = navegador.find_element(By.XPATH, '//*[@id="selDe"]')
                            select = Select(select_element)
                            select.select_by_visible_text('ANATEL/E-mail de replicação <nao-responda@anatel.gov.br>')
                            navegador.find_element(By.XPATH, '//*[@id="s2id_autogen1"]').send_keys(emailSol)
                            time.sleep(0.7)
                            navegador.find_element(By.XPATH, '//*[@id="select2-result-label-2"]').click()
                            navegador.find_element(By.ID, 'txtAssunto').send_keys(f'Processo SEI nº {processosAssinados} - Aprovado')
                            #DIFERE O EMAIL PARA PRODUTO RETIDO E NAO RETIDO
                            if not codigo_rastreioConforme.strip():
                                navegador.find_element(By.ID, 'txaMensagem').send_keys(textoNaoRetido)
                                prod = 'naoRetido'

                            else:
                                navegador.find_element(By.ID, 'txaMensagem').send_keys(textoRetido)
                                prod = 'Retido'

                            #PEDE PARA O USUARIO CONFIRMAR AS INFORMACOES
                            verifica = int(input('Verifique se está tudo nos conformes, se sim, digite [1], caso não, digite [2]: '))
                            if verifica == 1:
                                navegador.find_element(By.XPATH, '//*[@id="divInfraBarraComandosInferior"]/button[1]').click()
                                alert = Alert(navegador)
                                alert.accept()
                                navegador.switch_to.window(janela_principal)
                                navegador.switch_to.default_content()
                                #RETORNA PARA PAGINA INICIAL DO PROCESSO
                                navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processosAssinados)
                                elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                                elementos.send_keys(Keys.ENTER)
                                time.sleep(1)
                                #APAGA ANOTACAO DE AGUARDANDO ASSINATURA
                                navegador.switch_to.frame('ifrVisualizacao')
                                navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[17]').click()
                                navegador.find_element(By.XPATH, '//*[@id="txaDescricao"]').clear()
                                navegador.find_element(By.XPATH, '//*[@id="divInfraBarraComandosSuperior"]/button').click()
                                #INSERE TAG NO PROCESSO
                                navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[25]').click()
                                time.sleep(0.5)
                                navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
                                time.sleep(0.5)
                                #DIFERE A TAG PARA RETIDO E NAO RETIDO
                                if prod == 'naoRetido':
                                    navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/ul/li[16]').click()
                                else:
                                    navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/ul/li[17]').click()
                                #CONCLUI PROCESSO
                                navegador.find_element(By.XPATH, '//*[@id="sbmSalvar"]').click()
                                navegador.switch_to.default_content()
                                navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processosAssinados)
                                elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                                elementos.send_keys(Keys.ENTER)
                                time.sleep(1)
                                navegador.switch_to.frame('ifrVisualizacao')
                                navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[20]').click()
                                time.sleep(0.3)
                                navegador.switch_to.default_content()
                                #APAGA PROCESSO DO TXT DE PROCESSOS CONFORMES
                                with open('processosConformes.txt', 'r') as file:
                                    lines = file.readlines()

                                with open('processosConformes.txt', 'w') as file:
                                    for line in lines:
                                        if line.strip() != processosAssinados.strip():
                                            file.write(line)
                            #CASO HAJA ERRO NO EMAIL ELE CANCELA O ENVIO
                            else:
                                navegador.find_element(By.XPATH, '//*[@id="btnCancelar"]')
                                print(f"Confira o processo {processosAssinados} mais tarde.")
                                navegador.switch_to.window(janela_principal)
                                navegador.switch_to.default_content()
                        
                        #CASO O DESPACHO NAO ESTEJA ASSINADO ELE PEDE PARA AGUARDAR
                        else:
                            print(f"O despacho do processo {processosAssinados} ainda não foi assinado, execute o script novamente após conter a assinatura do gerente.")
                            time.sleep(0.5)
                #FLUXO DE DRONES IMPORTADOS PRA USO PROPRIO           
                elif check_element_exists(By.XPATH, '/html/body/table[5]/tbody/tr/td[2]') and tabela_cod != 'Código de rastreio':
                    codigo_rastreioConforme = navegador.find_element(By.XPATH, '/html/body/table[5]/tbody/tr/td[2]').text
                    emailSol = navegador.find_element(By.XPATH, '/html/body/div[3]/a[1]').text
                    navegador.switch_to.default_content()
                    navegador.switch_to.frame('ifrArvore')
                    #VERIFICA ASSINATURA DO DESPACHO DECISORIO
                    if check_element_exists(By.PARTIAL_LINK_TEXT, "Despacho Decisório"):
                        navegador.find_element(By.PARTIAL_LINK_TEXT, "Despacho Decisório").click()
                        navegador.switch_to.default_content()
                        navegador.switch_to.frame('ifrVisualizacao')
                        navegador.switch_to.frame('ifrArvoreHtml')
                        if check_element_exists(By.XPATH, "/html/body/div[1]/table[1]"):
                            #ENVIA EMAIL DE APROVACAO DO PROCESSO
                            navegador.switch_to.default_content()
                            navegador.switch_to.frame('ifrVisualizacao')
                            navegador.find_element(By.XPATH, "//img[@title='Enviar Documento por Correio Eletrônico']").click()
                            time.sleep(0.5)
                            navegador.switch_to.window(navegador.window_handles[-1])
                            time.sleep(0.5)
                            select_element = navegador.find_element(By.ID, 'selDe')
                            select = Select(select_element)
                            select.select_by_visible_text('ANATEL/E-mail de replicação <nao-responda@anatel.gov.br>')
                            navegador.find_element(By.XPATH, '//*[@id="s2id_autogen1"]').send_keys(emailSol)
                            time.sleep(0.5)
                            navegador.find_element(By.XPATH, '//*[@id="select2-result-label-2"]').click()
                            navegador.find_element(By.ID, 'txtAssunto').send_keys(f'Processo SEI nº {processosAssinados} - Aprovado')
                            #DIFERE O EMAIL PARA PRODUTO RETIDO E NAO RETIDO
                            if not codigo_rastreioConforme.strip():
                                navegador.find_element(By.ID, 'txaMensagem').send_keys(textoNaoRetido)
                                prod = 'naoRetido'
                            else:
                                navegador.find_element(By.ID, 'txaMensagem').send_keys(textoRetido)
                                prod = 'Retido'
                            
                            
                            #PEDE PARA O USUARIO CONFIRMAR AS INFORMACOES
                            verifica = int(input('Verifique se está tudo nos conformes, se sim, digite [1], caso não, digite [2]: '))
                            if verifica == 1:
                                navegador.find_element(By.XPATH, '//*[@id="divInfraBarraComandosSuperior"]/button[1]').click()
                                alert = Alert(navegador)
                                alert.accept()      
                                navegador.switch_to.window(janela_principal)
                                navegador.switch_to.default_content()
                                #RETORNA PARA PAGINA INICIAL DO PROCESSO
                                navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processosAssinados)
                                elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                                elementos.send_keys(Keys.ENTER)
                                time.sleep(1)
                                #APAGA ANOTACAO DE AGUARDANDO ASSINATURA
                                navegador.switch_to.frame('ifrVisualizacao')
                                navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[17]').click()
                                navegador.find_element(By.XPATH, '//*[@id="txaDescricao"]').clear()
                                navegador.find_element(By.XPATH, '//*[@id="divInfraBarraComandosSuperior"]/button').click()
                                #INSERE TAG NO PROCESSO
                                navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[25]').click()
                                time.sleep(1)
                                navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/div/a').click()
                                time.sleep(0.5)
                                #DIFERE A TAG PARA RETIDO E NAO RETIDO
                                if prod == 'naoRetido':
                                    navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/ul/li[16]').click()                            
                                else:
                                    navegador.find_element(By.XPATH, '//*[@id="selMarcador"]/ul/li[17]').click()
                                #CONCLUI PROCESSO
                                navegador.find_element(By.XPATH, '//*[@id="sbmSalvar"]').click()
                                navegador.switch_to.default_content()
                                time.sleep(0.3)
                                navegador.find_element(By.ID,'txtPesquisaRapida').send_keys(processosAssinados)
                                elementos = navegador.find_element(By.ID,'txtPesquisaRapida')
                                elementos.send_keys(Keys.ENTER)
                                time.sleep(1)
                                navegador.switch_to.frame('ifrVisualizacao')
                                navegador.find_element(By.XPATH, '//*[@id="divArvoreAcoes"]/a[20]').click()
                                time.sleep(0.3)
                                navegador.switch_to.default_content()
                                #APAGA PROCESSO DO TXT DE PROCESSOS CONFORMES
                                with open('processosConformes.txt', 'r') as file:
                                    lines = file.readlines()

                                with open('processosConformes.txt', 'w') as file:
                                    for line in lines:
                                        if line.strip() != processosAssinados.strip():
                                            file.write(line)
                            #CASO HAJA ERRO NO EMAIL ELE CANCELA O ENVIO
                            elif verifica == 2:
                                navegador.find_element(By.XPATH, '//*[@id="btnCancelar"]')
                                print(f"Confira o processo {processosAssinados} mais tarde.")
                                navegador.switch_to.window(janela_principal)
                                navegador.switch_to.default_content()
                            
                        #CASO O DESPACHO NAO ESTEJA ASSINADO ELE PEDE PARA AGUARDAR
                        else:
                            print(f"O despacho do processo {processosAssinados} ainda não foi assinado, execute o script novamente após conter a assinatura do gerente.")
                            time.sleep(0.5)
                break


In [9]:
#EXECUCAO PRINCIPAL DO CODIGO
while True:
    try:
        opcoes = int(input("O que deseja fazer?\nDigite [1] para analisar processo e criar despacho.\nDigite [2] para concluir processos assinados.\nDigite [3] para cancelar a operação. "))
        if opcoes == 1:
            analisaProcesso()
            print("Análise finalizada.")
        elif opcoes == 2:
            concluiProcesso()
        else:
            print("Encerrando programa")
            navegador.close()
            break
    except ValueError:
        print("Opção inválida, tente novamente.")


--------------------------------------------------------------------------
Processo: 53500.046023/2024-71
Interessado:  CARLOS ALBERTO ROSA
Solicitante:  Carlos Alberto Rosa


Peticionamento em interesse próprio como Pessoa Física.


+---------+----------------+-----------------------------------------------------+
| Modelo  | Nome Comercial | Número de Série (incluindo rádio controle e óculos) |
+---------+----------------+-----------------------------------------------------+
| DSDR04C |   ATOM DRONE   |             2AYUO-DSDR04B/2AYOU-DSRC02A             |
+---------+----------------+-----------------------------------------------------+


O modelo DSDR04C não se encontra na lista de drones conformes


Produto não retido ou código de rastreio não informado.


Confira o relatório fotográfico e veja se os documentos estão conformes!

Preencha planilha no sharepoint sobre o processo.
--------------------------------------------------------------------------
Tome as medidas necessárias 